In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import json
from pathlib import Path

In [3]:
path = Path('/data/google_nq/v1.0/train')
assert path.exists()

In [4]:
!ls -la {path}

total 89698580
drwxr-xr-x 2 root root       4096 Aug 25 19:30 .
drwxr-xr-x 5 root root       4096 Aug 25 16:07 ..
-rw-r--r-- 1 root root 5541577648 Aug 25 17:16 nq-train-00.jsonl
-rw-r--r-- 1 root root 5770348954 Aug 25 17:17 nq-train-01.jsonl
-rw-r--r-- 1 root root 5717125957 Aug 25 17:17 nq-train-02.jsonl
-rw-r--r-- 1 root root 5721671722 Aug 25 17:17 nq-train-03.jsonl
-rw-r--r-- 1 root root 5748361602 Aug 25 17:16 nq-train-04.jsonl
-rw-r--r-- 1 root root 5634916761 Aug 25 17:15 nq-train-05.jsonl
-rw-r--r-- 1 root root 5578705699 Aug 25 17:15 nq-train-06.jsonl
-rw-r--r-- 1 root root 5392278572 Aug 25 17:15 nq-train-07.jsonl
-rw-r--r-- 1 root root 5827270393 Aug 25 17:16 nq-train-08.jsonl
-rw-r--r-- 1 root root 5693204718 Aug 25 17:17 nq-train-09.jsonl
-rw-r--r-- 1 root root  876311133 Aug 25 17:17 nq-train-10.jsonl.gz
-rw-r--r-- 1 root root  878127326 Aug 25 17:16 nq-train-11.jsonl.gz
-rw-r--r-- 1 root root  889257016 Aug 25 17:17 nq-train-12.jsonl.gz
-rw-r--r-- 1 root root  89176912

In [5]:
dataset_files = [path/f for f in ['nq-train-00.jsonl', 'nq-train-01.jsonl', 'nq-train-02.jsonl', 'nq-train-03.jsonl', 'nq-train-04.jsonl', 'nq-train-05.jsonl', 'nq-train-06.jsonl', 'nq-train-07.jsonl', 'nq-train-08.jsonl', 'nq-train-09.jsonl']]
dataset_file = dataset_files[0]

In [6]:
!du -h {dataset_file}

5.2G	/data/google_nq/v1.0/train/nq-train-00.jsonl


In [7]:
!wc -l {dataset_file}

5961 /data/google_nq/v1.0/train/nq-train-00.jsonl


In [8]:
!head -n 1 {dataset_file} > {path/'single_line.json'}

In [9]:
with open(path/'single_line.json') as f:
    line = f.readline()

In [10]:
line_dict = json.loads(line)

In [11]:
line_dict.keys()

dict_keys(['annotations', 'document_html', 'document_title', 'document_tokens', 'document_url', 'example_id', 'long_answer_candidates', 'question_text', 'question_tokens'])

In [12]:
line_dict['question_text']

'when is the last episode of season 8 of the walking dead'

In [13]:
line_dict['annotations']

[{'annotation_id': 6782080525527814293,
  'long_answer': {'candidate_index': 92,
   'end_byte': 96948,
   'end_token': 3538,
   'start_byte': 82798,
   'start_token': 2114},
  'short_answers': [{'end_byte': 96731,
    'end_token': 3525,
    'start_byte': 96715,
    'start_token': 3521}],
  'yes_no_answer': 'NONE'}]

In [14]:
# line_dict['document_tokens']

In [15]:
line_dict['annotations']

[{'annotation_id': 6782080525527814293,
  'long_answer': {'candidate_index': 92,
   'end_byte': 96948,
   'end_token': 3538,
   'start_byte': 82798,
   'start_token': 2114},
  'short_answers': [{'end_byte': 96731,
    'end_token': 3525,
    'start_byte': 96715,
    'start_token': 3521}],
  'yes_no_answer': 'NONE'}]

In [16]:
# with open('sample.html', 'w') as f:
#     f.write(json.loads(full_data[5])['document_html'])

In [17]:
def extract_answer(line_dict):
    answer_set = set([])
    for la in line_dict['annotations']:
        long_answer = la['long_answer']
        start_byte = long_answer['start_byte']
        end_byte = long_answer['end_byte']
        long_answer_content = ""
        for l in line_dict['document_tokens']:
            if l['start_byte'] >= start_byte and l['end_byte'] <= end_byte and l['html_token'] is False:
                 long_answer_content += l['token'] + " "
        short_answers = la['short_answers']
        for short_answer in short_answers:
            start_byte = short_answer['start_byte']
            end_byte = short_answer['end_byte']
            answer = ""
            for l in line_dict['document_tokens']:
                if l['start_byte'] >= start_byte and l['end_byte'] <= end_byte and l['html_token'] is False:
                    answer += l['token'] + " "
            answer_set.add(answer.strip())
    
    return [(line_dict['question_text'] + '?', a, {'answer_start': long_answer_content.find(a), 'answer_end': long_answer_content.find(a) + len(a)}, long_answer_content) for a in answer_set if long_answer_content.find(a) > -1]

extract_answer(line_dict)

[('when is the last episode of season 8 of the walking dead?',
  'March 18 , 2018',
  {'answer_start': 6052, 'answer_end': 6067},
  "No . overall No. in season Title Directed by Written by Original air date U.S. viewers ( millions ) 100 `` Mercy '' Greg Nicotero Scott M. Gimple October 22 , 2017 ( 2017 - 10 - 22 ) 11.44 Rick , Maggie , and Ezekiel rally their communities together to take down Negan . Gregory attempts to have the Hilltop residents side with Negan , but they all firmly stand behind Maggie . The group attacks the Sanctuary , taking down its fences and flooding the compound with walkers . With the Sanctuary defaced , everyone leaves except Gabriel , who reluctantly stays to save Gregory , but is left behind when Gregory abandons him . Surrounded by walkers , Gabriel hides in a trailer , where he is trapped inside with Negan . 101 `` The Damned '' Rosemary Rodriguez Matthew Negrete & Channing Powell October 29 , 2017 ( 2017 - 10 - 29 ) 8.92 Rick 's forces split into separat

In [18]:
def navigate_data(full_data):
    answer_ds = []
    for i, l in enumerate(full_data):
        json_data = json.loads(l)
        for extracted in extract_answer(json_data):
            answer_ds.append(extracted)
    return [a for a in answer_ds if len(a[1]) > 0]
            


In [19]:
def check_ds(answer_ds):
    count_not_found = []
    for i, a in enumerate(answer_ds):
        if a[2]['answer_start'] < 0:
            count_not_found.append((i, a))

    assert len(count_not_found) == 0

In [20]:
%%time

full_data = []
answer_datasets = []
for dataset_f in dataset_files:
    print(f'Processing {dataset_f}')
    with open(dataset_f) as f:
        full_data = f.readlines()
        answer_ds = navigate_data(full_data)
        check_ds(answer_ds)
        answer_datasets += answer_ds

Processing /data/google_nq/v1.0/train/nq-train-00.jsonl
Processing /data/google_nq/v1.0/train/nq-train-01.jsonl
Processing /data/google_nq/v1.0/train/nq-train-02.jsonl
Processing /data/google_nq/v1.0/train/nq-train-03.jsonl
Processing /data/google_nq/v1.0/train/nq-train-04.jsonl
Processing /data/google_nq/v1.0/train/nq-train-05.jsonl
Processing /data/google_nq/v1.0/train/nq-train-06.jsonl
Processing /data/google_nq/v1.0/train/nq-train-07.jsonl
Processing /data/google_nq/v1.0/train/nq-train-08.jsonl
Processing /data/google_nq/v1.0/train/nq-train-09.jsonl
CPU times: user 8min 45s, sys: 22.5 s, total: 9min 7s
Wall time: 9min 58s


In [21]:
len(answer_datasets)

26121

In [22]:
import pickle

pickle.dump( answer_datasets, open( "answers.pkl", "wb" ) )

In [23]:
!du -h answers.pkl

27M	answers.pkl
